In [6]:
import sys
sys.path.append(".\\RNNsearch")

from train import *
from RNNsearch.RNNsearch import *
from RNNsearch.Encoder_search import *
from RNNsearch.Decoder_search import *
from RNNsearch.Allignement import *
from datasets import  load_from_disk
import torch
import torch.nn as nn

# Getting the vocabs

In [2]:
fr_vocab_path = '../../30k_fr.txt'  # Path to the French vocab file
with open(fr_vocab_path, 'r') as file:
    fr_words = [line.strip() for line in file]

# Cleaning words and creating the dictionary
word_dict_fr = {word: i for i, word in enumerate(fr_words)}

eng_vocab_path= '../../30k_eng.txt'  # Path to the English vocab file
# Open the file and read lines into a list
with open(eng_vocab_path, 'r') as file:
    eng_words = [line.strip() for line in file]

word_dict_eng = { word.strip() : i for i, word in enumerate(eng_words)}

In [5]:
print(len(word_dict_eng))
print(len(word_dict_fr))

30000
30000


# Initializing the model

In [3]:
#hyperparameters
batch_size = 80       # Number of sequences in a mini-batch
vocab_size = 30000     # Size of the input vocabulary
hidden_size = 1000     # Number of features in the hidden state
embedding_dim = 620   # Word embedding dimension
maxout_units= 500     # Number of units in the maxout layer
allign_dim=50        # Number of features in the allignment model Tx

In [5]:
encoder=Encoder(vocab_size,hidden_size,embedding_dim)
decoder=Decoder(vocab_size,hidden_size,embedding_dim,maxout_units,allign_dim)
model=RNNsearch(encoder,decoder,device='cuda')

In [7]:
def Init_weights(self):
        for name, param in model.named_parameters():

            if 'weight_hh' in name:
                nn.init.orthogonal_(param.data) 
            elif 'weight' in name:  
                nn.init.normal_(param.data, mean=0, std=0.01)  
            else:
                nn.init.constant_(param.data,0)

model.apply(Init_weights)

RNNsearch(
  (encoder): Encoder(
    (embedding): Embedding(30000, 620)
    (bi_RNN): GRU(620, 1000, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (Allignement): Allignement(
      (Wa): Linear(in_features=1000, out_features=50, bias=False)
      (Ua): Linear(in_features=2000, out_features=50, bias=False)
      (va): Linear(in_features=50, out_features=1, bias=False)
    )
    (embedding): Embedding(30000, 620)
    (gru): GRU(3620, 1000, batch_first=True)
    (maxout): Maxout(
      (lin): Linear(in_features=3620, out_features=1000, bias=True)
    )
    (fc): Linear(in_features=500, out_features=30000, bias=True)
  )
)